In [18]:

import numpy as np
import pandas as pd
from pathlib import Path
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import datetime as dt
from prophet import Prophet

In [3]:
df=pd.read_csv(Path('../resources/Crime_Data_from_2020_to_Present.csv'))

In [4]:
df.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,10304468,01/08/2020 12:00:00 AM,01/08/2020 12:00:00 AM,2230,3,Southwest,377,2,624,BATTERY - SIMPLE ASSAULT,...,AO,Adult Other,624.0,NaN,NaN,NaN,1100 W 39TH PL,NaN,34.0141,-118.2978
1,190101086,01/02/2020 12:00:00 AM,01/01/2020 12:00:00 AM,330,1,Central,163,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,700 S HILL ST,NaN,34.0459,-118.2545
2,200110444,04/14/2020 12:00:00 AM,02/13/2020 12:00:00 AM,1200,1,Central,155,2,845,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,...,AA,Adult Arrest,845.0,NaN,NaN,NaN,200 E 6TH ST,NaN,34.0448,-118.2474
3,191501505,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,1730,15,N Hollywood,1543,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),...,IC,Invest Cont,745.0,998.0,NaN,NaN,5400 CORTEEN PL,NaN,34.1685,-118.4019
4,191921269,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,415,19,Mission,1998,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,IC,Invest Cont,740.0,NaN,NaN,NaN,14400 TITUS ST,NaN,34.2198,-118.4468


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752911 entries, 0 to 752910
Data columns (total 28 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DR_NO           752911 non-null  int64  
 1   Date Rptd       752911 non-null  object 
 2   DATE OCC        752911 non-null  object 
 3   TIME OCC        752911 non-null  int64  
 4   AREA            752911 non-null  int64  
 5   AREA NAME       752911 non-null  object 
 6   Rpt Dist No     752911 non-null  int64  
 7   Part 1-2        752911 non-null  int64  
 8   Crm Cd          752911 non-null  int64  
 9   Crm Cd Desc     752911 non-null  object 
 10  Mocodes         649650 non-null  object 
 11  Vict Age        752911 non-null  int64  
 12  Vict Sex        654681 non-null  object 
 13  Vict Descent    654675 non-null  object 
 14  Premis Cd       752902 non-null  float64
 15  Premis Desc     752476 non-null  object 
 16  Weapon Used Cd  261472 non-null  float64
 17  Weapon Des

In [6]:
df.columns

Index(['DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA', 'AREA NAME',
       'Rpt Dist No', 'Part 1-2', 'Crm Cd', 'Crm Cd Desc', 'Mocodes',
       'Vict Age', 'Vict Sex', 'Vict Descent', 'Premis Cd', 'Premis Desc',
       'Weapon Used Cd', 'Weapon Desc', 'Status', 'Status Desc', 'Crm Cd 1',
       'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'LOCATION', 'Cross Street', 'LAT',
       'LON'],
      dtype='object')

In [7]:
code_crime=df[["Crm Cd", "Crm Cd Desc"]]
code_crime

,Crm Cd,Crm Cd Desc
0,624,BATTERY - SIMPLE ASSAULT
1,624,BATTERY - SIMPLE ASSAULT
2,845,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE
3,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER)
4,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA..."
...,...,...
752906,662,"BUNCO, GRAND THEFT"
752907,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA..."
752908,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT"
752909,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT"


In [8]:
code_crime["Crm Cd"].unique()

array([624, 845, 745, 740, 121, 442, 946, 341, 330, 930, 648, 626, 440,
       354, 210, 230, 310, 510, 420, 761, 236, 662, 350, 860, 480, 623,
       956, 900, 888, 331, 901, 886, 421, 647, 940, 810, 922, 812, 220,
       625, 755, 649, 815, 251, 320, 850, 820, 668, 902, 664, 920, 343,
       437, 753, 928, 910, 760, 762, 661, 351, 821, 237, 903, 813, 666,
       627, 805, 763, 441, 122, 443, 450, 434, 410, 890, 520, 670, 951,
       660, 654, 352, 250, 110, 933, 652, 950, 231, 345, 822, 814, 932,
       622, 471, 235, 470, 921, 906, 433, 651, 943, 653, 436, 439, 806,
       949, 522, 446, 113, 438, 451, 485, 944, 954, 756, 942, 487, 473,
       347, 435, 880, 444, 475, 474, 931, 865, 349, 452, 353, 870, 924,
       840, 948, 884, 904, 830, 445, 432, 882], dtype=int64)

In [9]:
filtered_code_crime = code_crime.drop_duplicates(subset = "Crm Cd").sort_values('Crm Cd')
filtered_code_crime

,Crm Cd,Crm Cd Desc
3205,110,CRIMINAL HOMICIDE
18619,113,"MANSLAUGHTER, NEGLIGENT"
5,121,"RAPE, FORCIBLE"
1462,122,"RAPE, ATTEMPTED"
19,210,ROBBERY
...,...,...
17402,949,ILLEGAL DUMPING
3479,950,"DEFRAUDING INNKEEPER/THEFT OF SERVICES, OVER $..."
1912,951,"DEFRAUDING INNKEEPER/THEFT OF SERVICES, $950 &..."
28954,954,CONTRIBUTING


In [10]:
filtered_code_crime.to_csv("output.csv")

In [11]:
filtered_code_crime.reset_index(drop=True, inplace=True)
filtered_code_crime

,Crm Cd,Crm Cd Desc
0,110,CRIMINAL HOMICIDE
1,113,"MANSLAUGHTER, NEGLIGENT"
2,121,"RAPE, FORCIBLE"
3,122,"RAPE, ATTEMPTED"
4,210,ROBBERY
...,...,...
133,949,ILLEGAL DUMPING
134,950,"DEFRAUDING INNKEEPER/THEFT OF SERVICES, OVER $..."
135,951,"DEFRAUDING INNKEEPER/THEFT OF SERVICES, $950 &..."
136,954,CONTRIBUTING


In [20]:
np.arange(187,294)

array([187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199,
       200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212,
       213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225,
       226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238,
       239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251,
       252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264,
       265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277,
       278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290,
       291, 292, 293])

In [24]:
filtered_code_crime.loc[filtered_code_crime["Crm Cd"].isin(np.arange(187,294)),"type_of_crime"]="violent"

In [28]:
df.loc[df["Crm Cd"].isin(np.arange(187,294)),"type_of_crime"]="violent"

In [30]:
df

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON,type_of_crime
0,10304468,01/08/2020 12:00:00 AM,01/08/2020 12:00:00 AM,2230,3,Southwest,377,2,624,BATTERY - SIMPLE ASSAULT,...,Adult Other,624.0,NaN,NaN,NaN,1100 W 39TH PL,NaN,34.0141,-118.2978,NaN
1,190101086,01/02/2020 12:00:00 AM,01/01/2020 12:00:00 AM,330,1,Central,163,2,624,BATTERY - SIMPLE ASSAULT,...,Invest Cont,624.0,NaN,NaN,NaN,700 S HILL ST,NaN,34.0459,-118.2545,NaN
2,200110444,04/14/2020 12:00:00 AM,02/13/2020 12:00:00 AM,1200,1,Central,155,2,845,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,...,Adult Arrest,845.0,NaN,NaN,NaN,200 E 6TH ST,NaN,34.0448,-118.2474,NaN
3,191501505,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,1730,15,N Hollywood,1543,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),...,Invest Cont,745.0,998.0,NaN,NaN,5400 CORTEEN PL,NaN,34.1685,-118.4019,NaN
4,191921269,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,415,19,Mission,1998,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,Invest Cont,740.0,NaN,NaN,NaN,14400 TITUS ST,NaN,34.2198,-118.4468,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752906,231510379,05/29/2023 12:00:00 AM,05/25/2023 12:00:00 AM,1100,15,N Hollywood,1548,2,662,"BUNCO, GRAND THEFT",...,Invest Cont,662.0,NaN,NaN,NaN,5300 DENNY AV,NaN,34.1667,-118.3643,NaN
752907,231604807,01/27/2023 12:00:00 AM,01/26/2023 12:00:00 AM,1800,16,Foothill,1663,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,Invest Cont,740.0,NaN,NaN,NaN,12500 BRANFORD ST,NaN,34.2466,-118.4054,NaN
752908,231606525,03/22/2023 12:00:00 AM,03/22/2023 12:00:00 AM,1000,16,Foothill,1602,1,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",...,Invest Cont,230.0,NaN,NaN,NaN,12800 FILMORE ST,NaN,34.2790,-118.4116,violent
752909,231210064,04/12/2023 12:00:00 AM,04/12/2023 12:00:00 AM,1630,12,77th Street,1239,1,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",...,Invest Cont,230.0,NaN,NaN,NaN,6100 S VERMONT AV,NaN,33.9841,-118.2915,violent
